In [7]:
from models import DeepLinear
from dgl.data import CoraGraphDataset, CiteseerGraphDataset, PubmedGraphDataset
import torch.nn as nn
import torch
import torch.nn.functional as F
import dgl
from time import perf_counter

In [8]:
device = 'cpu'
activation = nn.ReLU()
epochs = 50
batch_size = 10000
lr = 0.02
loss_fn = nn.CrossEntropyLoss()
weight_decay = 5e-4

In [9]:
data = PubmedGraphDataset()
g = data[0]
g = g.to(device)
features = g.ndata['feat']
labels = g.ndata['label']
train_mask = g.ndata['train_mask']
val_mask = g.ndata['val_mask']
test_mask = g.ndata['test_mask']
in_feats = features.shape[-1]
n_classes = data.num_labels
n_edges = g.number_of_edges()
g = dgl.remove_self_loop(g)
g = dgl.add_self_loop(g)

  NumNodes: 19717
  NumEdges: 88651
  NumFeats: 500
  NumClasses: 3
  NumTrainingSamples: 60
  NumValidationSamples: 500
  NumTestSamples: 1000
Done loading data from cached files.


In [10]:
model = DeepLinear("SGC",{},in_feats, 16, n_classes, 2, 2, activation, dropout=0.2)
optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

In [11]:
def train(model, g, features, labels, train_mask, val_mask, test_mask):
    t = perf_counter()
    for epoch in range(epochs):
        model.train()
        logits = model(g, features)
        loss = loss_fn(logits[train_mask], labels[train_mask])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        model.eval()
        logits = model(g, features)
        train_acc = torch.sum(logits[train_mask].argmax(1) == labels[train_mask]).item() / train_mask.sum().item()
        val_acc = torch.sum(logits[val_mask].argmax(1) == labels[val_mask]).item() / val_mask.sum().item()
        test_acc = torch.sum(logits[test_mask].argmax(1) == labels[test_mask]).item() / test_mask.sum().item()
        print(f'Epoch {epoch + 1:02d}, Loss: {loss:.4f}, Train: {train_acc:.4f}, Val: {val_acc:.4f}, Test: {test_acc:.4f}')

    training_time = perf_counter()-t
    print(f'Training time: {training_time:.4f}s')

In [12]:
train(model, g, features, labels, train_mask, val_mask, test_mask)

Epoch 01, Loss: 1.1131, Train: 0.3333, Val: 0.2360, Test: 0.2230
Epoch 02, Loss: 1.3907, Train: 0.6667, Val: 0.6160, Test: 0.6060
Epoch 03, Loss: 0.7675, Train: 0.7500, Val: 0.6800, Test: 0.6840
Epoch 04, Loss: 0.8101, Train: 0.7333, Val: 0.6440, Test: 0.6860
Epoch 05, Loss: 0.8300, Train: 0.7667, Val: 0.6760, Test: 0.7060
Epoch 06, Loss: 0.7819, Train: 0.8667, Val: 0.7100, Test: 0.7260
Epoch 07, Loss: 0.6724, Train: 0.8667, Val: 0.7280, Test: 0.7190
Epoch 08, Loss: 0.5693, Train: 0.9000, Val: 0.7500, Test: 0.7240
Epoch 09, Loss: 0.5133, Train: 0.9000, Val: 0.7340, Test: 0.7420
Epoch 10, Loss: 0.4986, Train: 0.9000, Val: 0.7420, Test: 0.7500
Epoch 11, Loss: 0.4193, Train: 0.9167, Val: 0.7620, Test: 0.7550
Epoch 12, Loss: 0.3656, Train: 0.9333, Val: 0.7680, Test: 0.7530
Epoch 13, Loss: 0.3369, Train: 0.9333, Val: 0.7680, Test: 0.7590
Epoch 14, Loss: 0.2975, Train: 0.9333, Val: 0.7760, Test: 0.7590
Epoch 15, Loss: 0.2672, Train: 0.9167, Val: 0.7760, Test: 0.7520
Epoch 16, Loss: 0.2498, T